ITERATIVE FUNCTION

#We have the NU as the input, and it returns Y as the output. So we want to iterate that but changing the NU

In [4]:
import numpy as np
import copy
from itertools import product
import scipy as sp
import pylab as plt

from scipy.stats import gamma

from DMF_model import DMF_sim, DMF_parameters


def HRF(times):
    """ Return values for HRF at given times """
    # Gamma pdf for the peak
    peak_values = gamma.pdf(times, 6)
    # Gamma pdf for the undershoot
    undershoot_values = gamma.pdf(times, 12)
    # Combine them
    values = peak_values - 0.35 * undershoot_values
    # Scale max to 0.6
    return values / np.max(values) * 0.6


if __name__=='__main__':

    # DMF parameters
    P = {}
    P = DMF_parameters(P)
    P['sigma'] = 0.02  # no noise

    T  = 50     # simulation time
    dt = 1e-4   # integration step
    P['T'] = T

    # layer specific external input
    stim_start = int(10/dt)    # start of stimulation
    stim_end   = int(40/dt)    # end of stimulation
    
    # Define the possible values for each input
    values = [50, 100, 150, 200, 250, 300]

    # Create an array of all possible combinations for each input
    input_combinations = list(product(values, repeat=2))

    # Initialize arrays to store the results
    Y_results = []

    # Iterate through all combinations for each population
    for population_combinations in input_combinations:
        # Update the U array based on the current population_combinations
        U = np.zeros((int(T/dt), P['M']))
        for i, val in enumerate(population_combinations):
            U[:, i] = val

        # Simulate
        I, H, F = DMF_sim(U, P)     # I - input current, H - membrane potential, F - firing rate
        Y = I[int(1/dt):, :] - np.mean(I[int(0.5/dt):int(1/dt), :], axis=0)  # deviation from baseline during stimulation (and remove initial transient)
        Y = (Y - np.min(Y)) / (np.max(Y) - np.min(Y)) * 2 - 1  # normalize Y between -1 and 1

        # Append the Y values to the results
        Y_results.append(Y)
    
    Y_results_array = np.array(Y_results)
    print(Y_results_array)

    # Now, B_results contains the scaling factors for each population for all combinations of input values.
    #populations = np.array(['L23E', 'L23I', 'L4E', 'L4I', 'L5E', 'L5I', 'L6E', 'L6I'])
    #colors      = plt.cm.Spectral(np.linspace(0, 1, len(populations)))
    #plt.figure(figsize=(10, 6))
    #plt.gca().set_prop_cycle(plt.cycler('color', plt.cm.Spectral(np.linspace(0, 1, 8))))
    #plt.title('Y')
    #plt.plot(Y)
    #plt.xlim([0, len(Y)-1])
    # You can analyze or use these results as needed.


    """
    NU - the vector which will be the main parameters to be inferred.
    B  - the main observable statistic which will be used to infer the parameters.

    we now obtain B values for each population and not for each layer. 
    This should be enough for initial tests, later we could include the whole LBR model with a BOLD signal per layer.
    """


[[[ 0.08644256  0.37115031  0.1533527  ...  0.29699216  0.2792695
   -0.19287763]
  [ 0.09731962  0.29857814  0.3520237  ...  0.03523539  0.22870959
   -0.21909571]
  [ 0.02013572  0.26741812  0.28742219 ... -0.03070724  0.26072916
   -0.00108847]
  ...
  [-0.28268908  0.07126655 -0.08930547 ... -0.06937768 -0.28774726
   -0.19998992]
  [-0.25274676 -0.00442127 -0.17344271 ... -0.08399549 -0.39865807
   -0.00960045]
  [-0.1393983  -0.09752596 -0.33033358 ... -0.11605711 -0.43352095
   -0.00514607]]

 [[ 0.16311505  0.00623465  0.00809298 ...  0.05734624  0.1935754
    0.18692056]
  [ 0.14495142 -0.12304736  0.16193006 ...  0.09784939  0.26659002
    0.28500496]
  [ 0.18643858 -0.17585577  0.1902071  ... -0.05109513  0.25611174
    0.24158375]
  ...
  [ 0.47039403  0.26037835  0.6678437  ...  0.35292896  0.06730255
   -0.04198386]
  [ 0.39634573  0.31199803  0.44894043 ...  0.40918634 -0.0213578
   -0.02034622]
  [ 0.38343229  0.0692135   0.59852634 ...  0.24587337  0.22427725
   -0.049

In [ ]:
B_results

Current: NU, more combinations. input ranges from 0 to 300 in incrememnts of possibly 25mA? add noise with the same combinations, sample around the value of th einput (randomly sampled as a normal distribution), 

Alex's notes:
- The recording matrix needs three dimensions. Condition, Layer and Timestep of the simulation. 
- Start from 0 firing rate to 300. The intensity should be from 50 to 300 picoA.

- Simulating each condition severlat times.
- Randomize the input intensity around a normal distribution for each point. 
	- So if the algorithm says that the input for the function is 100, instead it is:
	- I = 100 +- Normal Dist around 100. The error is for us to decide. Think of it ourselves. Depends on what the length of the steps are. 